<a href="https://colab.research.google.com/github/danilo-uea/RNA-2021.1-AA2.1/blob/main/RNA_AA_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neurais Artificiais - Atividade Avaliativa 2.1
> Criado por: Danilo Frazão, Rafael Aragão Morais & Yasser Tuma




In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from prettytable import PrettyTable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Importando os dados

> O arquivo com os dados está armazenado em uma pasta do Google Drive, como é um arquivo em binario, iremos utilizar a biblioteca numpy para extrair os dados.



In [ ]:
def import_data(path):
  dt = np.dtype(float)
  arq = np.fromfile(path, dtype=dt)

  ret = np.empty((0, 3), float)

  for i in range(0, len(arq), 3):
    linha = np.array([[arq[i], arq[i+1], arq[i+2]]])
    ret = np.append(ret, linha, axis=0)

  return ret


In [ ]:
drivePath = "/content/drive/My Drive/Colab Notebooks/RNA/data/dataAll.txt"
data_All = import_data(drivePath)
data_All

array([[-363.7884,  244.1423,    0.    ],
       [ 328.7572, -256.7658,    1.    ],
       [-164.9827,  613.2164,    0.    ],
       ...,
       [ 872.4311,  669.1192,    1.    ],
       [ 822.6894, -702.6489,    1.    ],
       [-140.147 ,  734.0983,    0.    ]])

##Perceptron


> As funções abaixo compõem o treinamento, ajuste dos pesos, de um Perceptron



In [ ]:
def activationFunction(u, teta):
  if(u >= teta):
    return 1
  else:
    return 0

In [ ]:
def sumFunction(X, W):
  sum = 0
  for x,w in zip(X,W):
    sum += x*w
  return sum

In [ ]:
def weightAdjustment(W, n, e, X):
  delta = np.dot((n*e), X)
  new_weight = np.add(W, delta);
  return new_weight

In [ ]:
def epoch(inputData, W, n, teta):
  adjusts = 0;
  weights = []
  for idx, row in enumerate(inputData):
    sum = sumFunction(row[:-1], W)
    yp = activationFunction(sum, teta)
    if(row[-1] != yp):
      adjusts += 1;
      W = weightAdjustment(W, n, (row[-1] - yp), row[:-1])
      weights.append((idx, W))
  return (adjusts, weights, W)



In [ ]:
def perceptronLearning(inputData, W, maxEpoch, n, teta):
  epochCount = 0
  adjustCount = 0
  adjust = True
  while(adjust):
    epochCount += 1
    (numAdjusts, oldWeights, W) = epoch(inputData, W, n, teta)

    print("\nNumero de epocas: " + str(epochCount))
    print("Numero de ajustes: " + str(numAdjusts))
    
    adjustCount += numAdjusts
    
    if(numAdjusts == 0):
      adjust = False #nao houve alteracoes nos valores dos pesos
      print("Os pesos foram ajustados")
    else:
      print("Pesos antigos e a linha em que foram alterados")
      for weight in oldWeights:
        print(weight[1], " - linha: ", weight[0])
      if(epochCount == maxEpoch):
        print("O limite maximo de epocas foi alcancado e nao foi possivel ajustar os pessos corretamente")
        break

  print("\nPesos finais: ", W)
  print("Numero total de epocas: " + str(epochCount))
  print("Numero total de ajustes: " + str(adjustCount))
  return (W, epochCount, adjustCount)


In [ ]:
def addBias(inputData, bias):
  nd= np.insert(inputData, 2, bias, axis=1)
  return nd


## Parte I - Resolvendo um problema linearmente separavel



> Para esta parte, iremos utilizar o arquivo dataAll.txt;

> O neurônio perceptron terá as seguintes características:

*   Função de ativação degrau com limiar = 0,

*   Taxa de aprendizado = 0.1,

*   Os pesos das entradas inicialmente serão valores aleatórios no intervalo de -0.5 e 0.5 .








In [ ]:
n_DataAll = 0.1 # taxa de aprendizado
teta_DataAll = 0 #degrau
w_DataAll = np.random.uniform(-0.5, 0.5, 3) #min, max, numero_de_valores
print("Vetor de pesos iniciais: ")
print(w_DataAll)
data_All = addBias(data_All, 1)
print("\n\nDados de entrada com o vies: \n [x1, x2, vies, y] \n")
data_All

Vetor de pesos iniciais: 
[-0.49149664  0.13898449  0.33716801]


Dados de entrada com o vies: 
 [x1, x2, vies, y] 



array([[-363.7884,  244.1423,    1.    ,    0.    ],
       [ 328.7572, -256.7658,    1.    ,    1.    ],
       [-164.9827,  613.2164,    1.    ,    0.    ],
       ...,
       [ 872.4311,  669.1192,    1.    ,    1.    ],
       [ 822.6894, -702.6489,    1.    ,    1.    ],
       [-140.147 ,  734.0983,    1.    ,    0.    ]])

In [ ]:
(finalWeights, num_epochs, num_adjusts) = perceptronLearning(data_All, w_DataAll, 0, n_DataAll, teta_DataAll)

In [ ]:
#grafico
def plotLine(inputData, W):
  h = []
  v = []
  for row in inputData:
    h.append(row[0])
    v.append(row[1])
    if(row[-1] == 0):
     plt.scatter(row[0], row[1], color='red')
    else:
      plt.scatter(row[0], row[1], color='blue')
  plt.plot(h , v, 'o')

plotLine(data_All, finalWeights)

In [ ]:
#plt.plot(, y, 'o', color='black');
data_All[0]

array([-363.7884,  244.1423,    1.    ,    0.    ])

# Parte 2 - Experimentacao

> Para esta parte, iremos utilizar o arquivo data0.txt;

> O neurônio perceptron terá as seguintes características:

*   Função de ativação degrau com limiar = 0,

*   Taxa de aprendizado = {0.4, 0.1, 0.01},

*   Os pesos das entradas inicialmente serão valores aleatórios no intervalo de {(-100,100) (-0.5 e 0.5)} ,

*   Para cada combinação entre os possiveis valores da taxa de aprendizado e do intervalo inicial dos pesos, será realizada 10 interações.










In [ ]:
drivePath2 = "/content/drive/My Drive/Colab Notebooks/RNA/data/data0.txt"
data0 = import_data(drivePath2)
data0

array([[-0.00149369, -0.03581784,  0.        ],
       [ 0.01234945,  0.10249524,  0.        ],
       [ 0.13954598,  0.05607083,  0.        ],
       ...,
       [ 0.95751872,  1.0035204 ,  1.        ],
       [ 0.97981588,  1.00949493,  1.        ],
       [ 1.02021556,  0.86533059,  1.        ]])

In [ ]:
def perceptronLearningPart2(inputData, W, maxEpoch, n, teta):
  epochCount = 0
  adjustCount = 0
  adjust = True
  while(adjust):
    epochCount += 1
    (numAdjusts, oldWeights, W) = epoch(inputData, W, n, teta)
    
    adjustCount += numAdjusts
    
    if(numAdjusts == 0):
      adjust = False #nao houve alteracoes nos valores dos pesos
    else:
        if(epochCount == maxEpoch):
          break
  return (W, epochCount, adjustCount)


In [ ]:
def iteration(inputData, w_index, n, teta):
  minEpochs = 0
  adjusts = []
  inputData = addBias(inputData, 1)

  for i in range(10):
    if(w_index == 0):
      W = np.random.uniform(-100, 100, 3)
    else:
      W = np.random.uniform(-0.5, 0.5, 3)
    
    (finalWeights, num_epochs, num_adjusts) = perceptronLearningPart2(inputData, W, 0, n, teta)

    if(minEpochs > num_epochs or not adjusts):
      minEpochs = num_epochs

    adjusts.append(num_adjusts)

  return ((sum(adjusts) / 10), np.std(adjusts), minEpochs)


In [ ]:
PT = PrettyTable()

first_row = iteration(data0, 0, 0.4, 0)
second_row = iteration(data0, 1, 0.4, 0)
third_row = iteration(data0, 0, 0.1, 0)
fourth_row = iteration(data0, 1, 0.1, 0)
fifth_row = iteration(data0, 0, 0.01, 0)
sixth_row = iteration(data0, 1, 0.01, 0)

PT.field_names = ["Taxa de aprendizado", "Intervalo de pesos", "Media do numero de ajustes", "Desvio padrao do numero de ajustes", "Menor numero de epocas"]
PT.add_row(["n=0.4", "{-0.5, 0.5}", first_row[0], first_row[1], first_row[2]])
PT.add_row(["n=0.4", "{-100, 100}", second_row[0], second_row[1], second_row[2]])
PT.add_row(["n=0.1", "{-0.5, 0.5}", third_row[0], third_row[1], third_row[2]])
PT.add_row(["n=0.1", "{-100, 100}", fourth_row[0], fourth_row[1], fourth_row[2]])
PT.add_row(["n=0.01", "{-0.5, 0.5}", fifth_row[0], fifth_row[1], fifth_row[2]])
PT.add_row(["n=0.01", "{-100, 100}", sixth_row[0], sixth_row[1], sixth_row[2]])

print(PT)

+---------------------+--------------------+----------------------------+------------------------------------+------------------------+
| Taxa de aprendizado | Intervalo de pesos | Media do numero de ajustes | Desvio padrao do numero de ajustes | Menor numero de epocas |
+---------------------+--------------------+----------------------------+------------------------------------+------------------------+
|        n=0.4        |    {-0.5, 0.5}     |           227.0            |         171.97557966176475         |           1            |
|        n=0.4        |    {-100, 100}     |            7.4             |         2.5377155080899043         |           3            |
|        n=0.1        |    {-0.5, 0.5}     |           1438.6           |         450.66244573960233         |           9            |
|        n=0.1        |    {-100, 100}     |            10.9            |         5.3376024580330075         |           2            |
|        n=0.01       |    {-0.5, 0.5}     |    

# Comentar sobre a tabela

# Parte 3 - Validacao Houlout em Problema Nao-Linearmente Separavel

> Para esta parte, iremos utilizar o arquivo dataHoldout.txt;

> O neurônio perceptron terá as seguintes características:











In [ ]:
n = 0.1 #Taxa de aprendizado
teta = 0 #Degrau usado na função de ativação
w = np.random.uniform(-0.5, 0.5, (1, 3)) #Vetor de pesos inicial
print("Vetor inicial de pesos: {}\n".format(w[0]))
 
def funcao(u): #Função de ativação
  if(u >= teta):
    return 1
  else:
    return 0
 
i = 0
ult_indice = len(dados_All) - 1
encerrar = True
qtd_epocas = 0
total_ajustes = 0
 
while i <= ult_indice:
  
  if (i == 0): #Verifica a a quantidade de épocas
    qtd_epocas += 1
    print('Época: {}'.format(qtd_epocas))
  
  linha = dados_All[i]
  yd = linha[2]
  x = np.array([-1, linha[0], linha[1]])
 
  u = 0
  for j in range(3):
    u += (x[j] * w[-1][j])
 
  y = funcao(u) #Chama a função de ativação
  e = yd - y #Erro
 
  if (e != 0): #Há necessidade de aprendizagem
    print('Ajuste do peso: {} -> Exemplo: {}'.format(w[-1], i + 1))
    peso_novo = np.array([w[-1] + ((n * e) * x)]) #Delta
    w = np.append(w, peso_novo, axis=0) #Começa a próxima época
    encerrar = False
    total_ajustes += 1
 
  if (ult_indice == i): #Última linha
    print('Número de ajustes: {}'.format(total_ajustes))
    print('Vetor de pesos: ', w[-1])
    print('\n')
    if (encerrar == False):
      total_ajustes = 0
      encerrar = True
      i = -1 #Vai para o começo do array de dados
 
  i += 1
 
print('Total de ajustes: ', len(w) - 1)
print('Total de épocas: ', qtd_epocas)

Vetor inicial de pesos: [-0.01377581 -0.25390031 -0.20273453]

Época: 1
Ajuste do peso: [-0.01377581 -0.25390031 -0.20273453] -> Exemplo: 1
Ajuste do peso: [  0.08622419  36.12493969 -24.61696453] -> Exemplo: 4
Ajuste do peso: [   0.18622419  -17.53411031 -101.11271453] -> Exemplo: 8
Ajuste do peso: [  0.08622419  55.26190969 -65.28315453] -> Exemplo: 19
Ajuste do peso: [-1.37758150e-02  1.37860610e+02  1.45726755e+01] -> Exemplo: 27
Ajuste do peso: [ 8.62241850e-02  1.43706370e+02 -6.37219145e+01] -> Exemplo: 32
Ajuste do peso: [   0.18622419  103.69745969 -141.61460453] -> Exemplo: 44
Ajuste do peso: [  0.28622419 157.36726969 -92.52233453] -> Exemplo: 46
Ajuste do peso: [   0.18622419  104.40720969 -149.39179453] -> Exemplo: 48
Ajuste do peso: [  0.28622419 163.96672969 -99.26946453] -> Exemplo: 72
Ajuste do peso: [   0.18622419  118.44209969 -164.90539453] -> Exemplo: 108
Ajuste do peso: [  0.28622419 209.15795969 -90.19207453] -> Exemplo: 111
Ajuste do peso: [   0.18622419  151.37